In [1]:
# February 12st, 2020
# This script compares classification in each classification system
# Second level matches specifications of Nemet and Johnson 2012


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import seaborn as sns

import sys
sys.path.append('/home/rkogeyam/scripts/')
sys.path.append('scripts/')

from determinants_scripts import classes, dtypes

from classification import preprocessing

citation='data/cleanuspatentcitation.csv'

usecols=['uuid', 'patent_id', 'citation_id']
citation_df=pd.read_csv(citation, usecols=usecols)


In [3]:
%matplotlib inline

In [4]:
sns.set()

In [5]:
# class_systems=['wipo']
class_systems=['wipo', 'ipcr', 'cpc', 'nber']

In [6]:
# This approach uses too much memory
# Try something else

for class_system in class_systems:
    
    classification = 'data/'+class_system+'.csv'
    
    class_df=pd.read_csv(classification, dtype=object, usecols=[0,1,2])
    df=preprocessing(class_df, citation_df)
    df.rename(columns={df.columns[2]:'level1_pat', df.columns[3]:'level2_pat'}, inplace=True)
    
    df=preprocessing(class_df, df, generality=False)
   
    far_ext=class_system+'_far_ext'
    ext=class_system+'_ext'
    df[far_ext]=np.where(df['level1_pat'] == df['wipo_sector_id'], 1, 0)
    df[ext]=np.where(df['level2_pat'] == df['wipo_field_id'], 1, 0)
    
    if class_system=='wipo':
        output=df['uuid', far_ext, ext].set_index('uuid')
    else:
        output=output.join(df['uuid', far_ext, far_ext].set_index('uuid'))
        
print(output.head())

                               uuid wipolevel_1_pat wipolevel_2_pat  \
10001951  jskw4d0xoqe3929xt8wh5clrd               3              29   
10049948  gnj3nv25xw7tqd6b1ge3fite1               3              29   
3930271   b4h1nhpv805nbcft39sk2pmo4               4              33   
3930271   pt582ia7vdx6rmr9yqi4ewall               4              33   
3930271   owlcbgprjwhepmmlve0q1iuqc               4              33   

         wipo_sector_id wipo_field_id  wipo_far_external  wipo_external  
10001951              1            06                  0              0  
10049948              1            08                  0              0  
3930271               4            33                  1              1  
3930271               4            33                  1              1  
3930271               4            33                  1              1  
